In [1]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
#!pip install mnist
import mnist
%matplotlib inline
plt.style.use('default')

# load data
train_images = mnist.train_images()
train_labels = mnist.train_labels()
num_train_images = len(train_labels)
test_images = mnist.test_images()
test_labels = mnist.test_labels()
num_test_images = len(test_images)

# print the data dimensions
print("Train Images Shape: "+str(train_images.shape))
print("Train Labels Shape: "+str(train_labels.shape))
print('Train Images DataType: '+str(train_images.dtype))
print("Test Images Shape: "+str(test_images.shape))
print("Test Labels Shape: "+str(test_labels.shape))
print('Test Images DataType: '+str(test_images.dtype))

Train Images Shape: (60000, 28, 28)
Train Labels Shape: (60000,)
Train Images DataType: uint8
Test Images Shape: (10000, 28, 28)
Test Labels Shape: (10000,)
Test Images DataType: uint8


In [2]:
# define the nonlinear ReLU function
def ReLU(x):
  return np.max((0,x))

In [3]:
# define a layer class
class FClayer:
  
  def __init__(self, numInputs, numNodes):
    self.numInputs = numInputs
    self.numNodes = numNodes
    self.weights = np.random.normal(0.0, 0.05, size=(numNodes, numInputs))
    self.biases = np.zeros(numNodes)

  def apply(self, inputs):
    if (len(inputs) != self.numInputs):
      # check here so if there is an error we get told nicely about it.
      print("WARNING: Inputs to layer wrong size for the layer.")
      output = np.zeros(self.numNodes)
      return output
    # create a vector to hour the output of the layer
    output = np.zeros(self.numNodes)
    # Apply weights and baises
    x = np.matmul(self.weights, inputs) + self.biases
    # Apply the nonlinear function to each node
    for i in range(self.numNodes):
      output[i] = ReLU(x[i])
    return output

In [4]:
# define the softmax function.  
# This converts a vector of K real numbers into a probability distribution of K possible outcomes. 
# It is a generalization of the logistic function to multiple dimensions, and used in multinomial logistic regression.
def softmax(inputs):
  outputs = np.exp(inputs)/np.sum(np.exp(inputs))
  return outputs

In [ ]:
# define two hidden layers and an output layer
layer1 = FClayer(28*28,30)
layer2 = FClayer(30,10)
layerFinal = softmax
'''
layer1 = FClayer(28*28, 250)
layer2 = FClayer(250, 200)
layer3 = FClayer(200, 150)
layer4 = FClayer(150, 100)
layer5 = FClayer(100, 50)
layer6 = FClayer(50, 10)
layerFinal = softmax
'''

In [ ]:
# derivative of Loss function with respect to the 10 outputs of the second hidden layer (h_0^2,...,h_9^2)
def dl_dh2(network,obs,y):
  yHat = network.apply(obs)
  numh2i = network.layer2.numNodes 
  dldh2 = np.zeros(numh2i)
  k = np.where(y == 1)
  for i in range(numh2i):
    dldh2[i] = yHat[i] - y[i]
  return dldh2

In [ ]:
def dL_dW2(network,obs,y):
  # the part in paranthesis (y1\hat{y1},...,ym\hat{ym})is dl_dh2:
  dldh2 = dl_dh2(network,obs,y)
  numh2i = network.layer2.numNodes 
  numh1i = network.layer1.numNodes 
  # define the variable to hold the output 
  # this is an array the same size as the weight matrix
  dLdW2 = np.zeros((numh2i,numh1i))
  for i in range(numh2i):
    for j in range(numh1i):
      if (network.out2[i] == 0):
        dLdW2[i,j] = 0
      else:
        dLdW2[i,j] = dldh2[i]*network.out1[j]
  return dLdW2

In [ ]:
def dL_db2(network,obs,y):
  # the part in paranthesis (y1\hat{y1},...,ym\hat{ym})is dl_dh2:
  dldh2 = dl_dh2(network,obs,y)
  numh2i = network.layer2.numNodes 
  # define the variable to hold the output 
  # this is a vector the same size as the number of nodes in layer 2
  dLdb2 = np.zeros(numh2i)
  for i in range(numh2i):
    if (network.out2[i] == 0):
      dLdb2[i] = 0
    else:
      dLdb2[i] = dldh2[i]
  return dLdb2

In [ ]:
def dL_dW1(network,obs,y):
  # the part in paranthesis (y1\hat{y1},...,ym\hat{ym})is dl_dh2:
  dldh2 = dl_dh2(network,obs,y)
  numh2i = network.layer2.numNodes 
  numh1i = network.layer1.numNodes 
  numInputs = network.layer1.numInputs
  # define the variable to hold the output 
  # this is an array the same size as the weight matrix
  dLdW1 = np.zeros((numh1i,numInputs))
  for i in range(numh1i):
    for k in range(numh2i):
      if not (network.out2[k] == 0) or (network.out1[i] == 0):
        for j in range(numInputs):
          dLdW1[i,j] = dLdW1[i,j] + dldh2[k]*network.layer2.weights[k,i]*obs[j]
  return dLdW1

In [ ]:
def dL_db1(network,obs,y):
  # the part in paranthesis (y1\hat{y1},...,ym\hat{ym})is dl_dh2:
  dldh2 = dl_dh2(network,obs,y)
  numh2i = network.layer2.numNodes 
  numh1i = network.layer1.numNodes 
  numInputs = network.layer1.numInputs
  # define the variable to hold the output 
  # this is an array the same size as the weight matrix
  dLdb1 = np.zeros((numh1i))
  for i in range(numh1i):
    for k in range(numh2i):
      if not (network.out2[k] == 0) or (network.out1[i] == 0):
        dLdb1[i] = dLdb1[i] + dldh2[k]*network.layer2.weights[k,i]
  return dLdb1

In [ ]:
# define the network (this time using the )
class network2layers():
  
  def __init__(self, layer1, layer2, layerFinal):
    self.layer1 = layer1
    self.layer2 = layer2
    self.layerFinal = layerFinal
  
  def apply(self, inputs):
    self.inputs = inputs
    # This determines the network 'architecture'
    self.out1 = self.layer1.apply(self.inputs)
    self.out2 = self.layer2.apply(self.out1)
    self.outFinal = self.layerFinal(self.out2)
    return self.outFinal

  def train(self, obs, y, alpha):
    #   obs = one observation (input into the network)
    #   y = the truth value for the obs
    #   alpha = the training rate
    self.layer1.weights = self.layer1.weights - alpha*dL_dW1(network,obs,y)
    self.layer1.biases = self.layer1.biases - alpha*dL_db1(network,obs,y)
    self.layer2.weights = self.layer2.weights - alpha*dL_dW2(network,obs,y)
    self.layer2.biases = self.layer2.biases - alpha*dL_db2(network,obs,y)
  
  def trainMiniBatch(self, obs_list, y_list, alpha):
    self.batch_size = len(y_list)
    deltaL_dW1 = 0
    deltaL_db1 = 0
    deltaL_dW2 = 0
    deltaL_db2 = 0
    for i in range(self.batch_size):
      y = y_list[i]
      obs = obs_list[i]
      deltaL_dW1 = deltaL_dW1 + dL_dW1(network,obs,y)
      deltaL_db1 = deltaL_db1 + dL_db1(network,obs,y)
      deltaL_dW2 = deltaL_dW2 + dL_dW2(network,obs,y)
      deltaL_db2 = deltaL_db2 + dL_db2(network,obs,y)
      #print(dL_dW1(network,obs,y))
    self.layer1.weights = self.layer1.weights -  alpha*deltaL_dW1/self.batch_size
    self.layer1.biases = self.layer1.biases - alpha*deltaL_db1/self.batch_size
    self.layer2.weights = self.layer2.weights - alpha*deltaL_dW2/self.batch_size
    self.layer2.biases = self.layer2.biases - alpha*deltaL_db2/self.batch_size
    #print(deltaL_dW1)
    #print(deltaL_db1)
    #print(deltaL_dW2)
    #print(deltaL_db2)

# create an instance of this network 
# (the weights will be randomly selected since it is not trained)
network = network2layers(layer1, layer2, layerFinal)

In [ ]:
# make a list to hold the loss as we train
L = []

In [ ]:
# define our cross entropy loss function
def Loss(y,yHat):
  L = 0
  m = len(y)
  for i in range(m):
    # modify yHat[i] so we don't get log of zero (undefined).
    yHat[i] = max(10**(-10),yHat[i])
    L = L + y[i]*np.log(yHat[i])
  return -L

In [ ]:
num_iters = 20
batch_size = 20
for i in range(num_iters):
  batch_idx = np.random.randint(0, high=num_train_images, size=batch_size)
  y_list = []
  obs_list = []
  for idx in batch_idx:
    y = np.zeros(10)
    y[train_labels[idx]] = 1
    y_list.append(y)
    obs = train_images[idx,:,:].flatten()/255.  
    obs_list.append(obs)
  network.trainMiniBatch(obs_list, y_list, 1)
  
  idx = random.randrange(num_test_images)
  y = np.zeros(10)
  y[test_labels[idx]] = 1
  obs = test_images[idx,:,:].flatten()/255.  
  yhat = network.apply(obs)
  L.append(Loss(y,yhat))
  print('Completed '+str(i)+' of '+str(num_iters), end = '')
  print(' -- Truth: '+str(test_labels[idx])+' | Predicted Prob: '+str(yhat[test_labels[idx]]))